# Tarefa 11 de Álgebra Linear Computacional

Atividade sobre Método QR para Autovalores e Autovetores.

* **Aluna:** Bárbara Neves
* **Matrícula:** 507526


### Descrição

Implemente o Método QR para achar os autovalores e autovetores de matrizes.
1.  Matrizes simétricas
  - Entre com uma matriz simétrica $A_{n \times n}$ e uma tolerância.
  - Aplique o Método QR diretamente sobre a matriz de entrada e encontre a matriz final (diagonal) contendo os autovalores e a matriz acumulada $\{Q = Q_1, Q_2, Q_3, Q_4, \cdots \}$ cujas colunas são os autovetores da matriz original.
  - Aplique o método de Householder primeiro (primeira caixa preta) e depois o método QR sobre a saída do Método de Householder, obtendo a matriz diagonal que contém os autovalores da matriz original. Obtenha os autovetores da matriz original como colunas da matriz $P = HQ$, onde $H$ é a matriz acumulada de Householder e $Q$ é a matriz acumulada no método $QR$.

2. Matrizes não simétricas
  - Repita o que foi feito para matrizes simétricas. Porém, a matriz final do método $QR$ é uma matriz Triangular Superior e Blocos (BUT).
  - Encontre os autovalores da matriz original, achando os autovalores dos blocos $2 \times 2$ ao longo da diagonal da BUT.
  - Ache os autovetores da matriz BUT e, em seguida, os autovetores da matriz original.

# Imports

In [1]:
import numpy as np
np.set_printoptions(precision=2, suppress=True)

import warnings
warnings.filterwarnings("ignore")

# Algoritmo QR para Autovalores

O **Método QR para Autovalores e Autovetores** utiliza as transformações de similaridade para encontrar os autovetores e autovalores de uma matriz.

Podem ser utilizadas **matrizes de Householder** juntamente com a **Decomposição QR**, como exemplificado abaixo, $k$ corresponde ao número de iterações necessárias para a conversão do algoritmo do **Método QR**:

\\

\begin{gather*}
D = Q_{k}^T Q_{k-1}^T ... Q_1^T (H_{n-2}^T ... H_1^T A H_1 ... H_{n-2}) Q_{1} ... Q_{k-1} Q_k
\end{gather*}

\\


---

\\

- **Particularidades:** 

> Se a matriz $A$ for simétrica, a matriz $D$ é diagonal e os autovetores da matriz original correspondem aos elementos dessa diagonal.
>
> Caso contrário, a matriz $D$ é uma matriz *Block Upper Triangular* (BUT).

A matriz de autovetores da matriz $\Phi$ original $A$ é obtida pelo acúmulo de todas as matrizes utilizadas nas transformações de similaridade. 

Assim, temos que:

\begin{align}
\Phi =  H_1 ... H_{n-2} Q_{1} ... Q_{k-1} Q_k
\end{align}

\\







In [2]:
# ------> Funções utilitárias de outros trabalhos <---------

# Construção da matriz de Householder
def build_H(A, j):
  A = A.copy()
  s = A.shape[0]
  v = A[:, j]
  v[0:j + 1] = 0
  w = np.zeros(s)
  w[j + 1] = np.linalg.norm(v)
  
  N = v - w
  n = N / np.linalg.norm(N)
  H = np.eye(s) - 2 * (n[:, None] @ n[None, :])
  
  return H

# Aplicação da transformação de similaridade
def householder(A):
  T = A.copy()
  n = T.shape[0]
  Hc = np.eye(n)
  
  for j in range(n - 2):
    H = build_H(T, j)
    T = H @ T @ H
    Hc = Hc @ H
  
  return Hc, T

# Construção da matriz de Householder modificadar para a Decomposição QR
def build_Q(A, j):
  A = A.copy()
  s = A.shape[0]
  v = A[:, j]
  v[0: j] = 0
  w = np.zeros(s)
  w[j] = np.linalg.norm(v)
  
  if v[j] > 0: # Verificação para evitar problemas numéricos
    w[j] = -np.linalg.norm(v)    
  
  N = v - w
  n = N / np.linalg.norm(N)
  H = np.eye(s) - 2 * (n[:, None] @ n[None, :])
  
  return H

def QR_decomposition(A):
  R = A.copy()
  n = R.shape[0]
  Q = np.eye(n)
  
  for j in range(n - 1):
    Qj = build_Q(R, j)
    R = Qj @ R
    Q = Q @ Qj
  
  return Q, R

In [3]:
def QR(A, is_symmetric=True, e=1e-10):
  n = A.shape[0]
  d_prev = 0
  d = 0
  P, T = householder(A) # Calculando o método de Householder previamente
  
  while True:
    d_prev = d
    Q, R = QR_decomposition(T)
    T = R @ Q
    P = P @ Q
    d = np.linalg.norm(T.diagonal())
    if (abs(d_prev - d) / d) < e:
      break
  
  # T = autovalores; P = autovetores
  # P é a matriz acumulada das transformações de similiridade e
    # contém a matriz original das multiplicações
  if is_symmetric:
    return T, P
  else:
    # eig_vals = eigenvalues(T)
    # eig_vecs = eigenvectors(T)
    # return eig_vals, P @ eig_vecs 
    raise BaseException('Formato inesperado da matriz.')

## Exemplos

Abaixo estão os mesmos exemplos que os da [Tarefa 08](https://drive.google.com/file/d/1qp7zh1aLuV5i1c5hOdD2dNuESuudJtH-/view?usp=sharing).

### Matriz $A_1$

A matriz exemplo foi retirada da seguinte [aula](https://https://www.youtube.com/watch?v=6StS7VjtuGI):

\\

\begin{gather*} 
  A_{1 \; 3 \times 3} = \begin{bmatrix}
  5 & -2 & -2 \\
  -3 & 5 & 0 \\
  23 & -19 & -6
  \end{bmatrix}
\end{gather*}

\\

Essa matriz possui **2 autovetores independentes, com autovalores associados a eles**.

\\

<center>
  <img src="https://drive.google.com/uc?id=1XnvCOP0DwuUW8H2ajRA3-LZBDaiyvP-n" />
</center>

\\

---

\\

### Matriz $A_2$

A matriz exemplo foi retirada do seguinte [*link*](https://bvanderlei.github.io/jupyter-guide-to-linear-algebra/Approximating_Eigenvalues.html):

\\

\begin{gather*} 
  A_{2 \; 3 \times 3} = \begin{bmatrix}
  9 & -1 & -3 \\
  0 & 6 & 0 \\
  -6 & 3 & 6
  \end{bmatrix}
\end{gather*}

\\

Essa matriz possui **3 autovetores independentes, com autovalores associados a eles**.

\\

<center>
  <img src="https://drive.google.com/uc?id=13WWijx3FMZbujD50YZ6Flt3Cc39yv6X-" />
</center>

\\

In [4]:
A_1 = np.array([
  [5, -2, -2],
  [-3, 5, 0],
  [23, -19, -6]
], dtype='float32')

A_2 = np.array([
  [9, -1, -3],
  [0, 6, 0],
  [-6, 3, 6]
], dtype='float32')

In [5]:
eigen_values_1, eigen_vectors_1 = QR(A_1)

print("Autovalores = \n{}\n\n Autovetores = \n{}".format(eigen_values_1, eigen_vectors_1))

Autovalores = 
[[ 2.   -2.01 10.14]
 [ 0.    1.   29.67]
 [-0.   -0.    1.  ]]

 Autovetores = 
[[-0.67  0.07 -0.74]
 [-0.67  0.39  0.64]
 [-0.33 -0.92  0.21]]


In [6]:
np.diagonal(eigen_values_1)

array([2., 1., 1.])

In [7]:
eigen_values_2, eigen_vectors_2 = QR(A_2)

print("Autovalores = \n{}\n\n Autovetores = \n{}".format(eigen_values_2, eigen_vectors_2))

Autovalores = 
[[12.   -3.    2.83]
 [ 0.    3.    1.41]
 [ 0.    0.    6.  ]]

 Autovetores = 
[[ 0.71 -0.71  0.  ]
 [-0.   -0.   -1.  ]
 [-0.71 -0.71  0.  ]]


In [8]:
np.diagonal(eigen_values_1)

array([2., 1., 1.])